# 画像処理と CNN の基礎

この章では、まずコンピュータがどのように画像を扱うかについて学びます。続いて、基本的な画像処理の操作を紹介した後、コンピュータビジョンの物体認識などのタスクに優れた、画像処理で用いられる**畳み込み演算**を元に考えられた**畳み込みニューラルネットワーク（convolutional neural network, 以下 CNN）**を紹介します。

## 画像処理の基礎

### コンピュータが扱う画像

コンピュータが扱う画像には、まず**高さ（height）**と**幅（width）**が存在します。最小単位は**ピクセル**です。例えば、縦 32 ピクセル、横 64 ピクセルの画像は、合計 $32\times64=2048$ のピクセルから成ります。加えて、奥行きを表す**チャンネル（channel）**が存在します。
チャンネルに関しては、赤（red）、緑（green）、青（blue）の 3 つの色を表現した **RGB** が一般的ですが、他に CMYK カラーと言った規格もあります。各ピクセルに、それぞれのチャンネルの値が紐付けられています。この値は**輝度**（明るさの度合）を示す数値で、 0~255 の非負の整数値で表すことができます。なぜ 0~255 で表現するかというと、1 バイトで表現できる値が 0~255 の合計 256 個となっていて、コンピュータで扱う以上各値が 1 バイトに収まるのが好都合であるためです。1 バイト = 8 ビットであり、2 進数で表すと $2^8=256$ ビットになります。下の図の画像は RGB の 3 つのチャンネルを持つ縦 5 ピクセル、横 5 ピクセルの画像です。この画像をコンピュータに保存するには、最低でも合計 $3\times5\times5=75$ バイトが必要になるわけです。

![画像の構成要素](images/10/01.png)

### フィルタと畳み込み演算

別名**カーネル**とも呼ばれる**フィルタ**は、画像の上を計算しながら移動させることで、画像を変換させたり、特徴を抽出するための、画像処理で用いられる配列です。
多くの場合、画像より小さく、$3\times3$ や $5\times5$ です。これを使って施す処理を畳み込み演算と呼びます。

画像の色調や明度を変化させたり、輪郭を目立たせるなどの変換を行う機能が、「フィルタ」という名前でスマートフォンの画像加工アプリなどで用意されていることがあります。ここで扱うフィルタは厳密には同じものではありません。しかし、画像に対して変換を行うという点では「フィルタ」に近いので、これから説明する畳み込み演算を理解するのに役立つかもしれません。

### 畳み込み演算の一例 ー エッジ検出

エッジ検出の例を通して畳み込み演算を学んでいきましょう。エッジとは、物体の境界線や端を意味し、エッジ検出はこれらを畳み込みにより取り出す操作です。特定サイズのフィルタ、この例では $3\times3$ を用意し、左上から右下に向けて画像全体に処理を施します。処理後、画像からエッジを検出したものが残ります。以下の図は、そのエッジ検出の畳み込み演算を行った際のイメージ図を描いています。処理前の画像を「オリジナル（ORIGINAL）画像」とします。

![フィルタ](images/10/02.png)

フィルタを使った畳み込み演算は非常に単純です。今回は、1 列目の値が -1 、2 列目の値が 0、3 列目の値が 1 となっているフィルタを使用します。以下の図で説明していきます。

![エッジ検出](images/10/03.png)

まず、オリジナル画像の左上にフィルタを重ね合わせ、それぞれ対応する（重なる）部分の数値を確認します。これらの重なり合う数値同士の積を求め、その総和をエッジ検出後の左上の値として残します。計算後、フィルタを 1 ピクセル右にずらし、再度重なり合う部分で同じ計算をし、その結果を先程計算した値の一つ右に書き込みます。これらの操作をフィルタの右端とオリジナル画像の右端が一致するまで繰り返します。その後、下に 1 ピクセルずらし、左端まで戻って同様の操作を行います。ここでは上下左右にフィルタを 1 ピクセルずつ移動しましたが、一度に移動するピクセル数は自由に指定することができます。これを**ストライド**と言います。この操作が畳み込みです。点線で囲まれた計算を式にすると次のようになります。

$$
\begin{align}
\begin{array} { r } { - 1 \times 0 + 0 \times 1 + 1 \times 1 } \\ { + ( - 1 ) \times 1 + 0 \times 2 + 1 \times 2 } \\ { + ( - 1 ) \times 3 + 0 \times 3 + 1 \times 1}\\
{ = 0 }\end{array}
\end{align}
$$

オリジナル画像のサイズがこの例のように $5\times5$ の場合、フィルタ処理後の画像のサイズは $3\times3$ になります（上の図では $5\times5$ になっていることに注意してください）。実際に $5\times5$ の画像に対して、$3\times3$ のフィルタを右端まで計算してみましょう。3 回ストライドを行った結果右端に辿り着き、それ以上ストライドできない状態となります。これは、上下左右 1 ピクセルずつ小さくした画像が残ることになります。一度のストライド幅を大きくすると、残る画像のサイズは更に小さくなります。このように、そのままフィルタをかけてしまうと、画像のサイズが変わってしまうため、以下のように周りに値を足してあげます。この処理を**パディング**と呼び、一般的には総和に影響しない 0 の値で埋めることが多いです。この処理を施すことにより、畳み込み演算後の画像サイズも一定に保つことがでます。

![Padding](images/10/04.png)

#### フィルタでなぜエッジが検出できるのか

エッジは、色の変化量が大きい部分と考えられます。異なる色同士の境界線にエッジがあると人は認識します。この変化量は数学でいう関数の勾配と同じで微分することによって得ることができます。これが上のエッジ検出のフィルタの値とどう関係しているのかを分析します。

まず、以下のような横軸 $x$、縦軸 $f(x)$ のグラフを考えます。$x$ はあるピクセルの位置、$f(x)$ はその $x$ における輝度です。畳み込み計算を行うとき、 $x$ は離散的で、 $f(x)$ も故に滑らかな関数ではないですが、局所的に見た時に現実の世界での輝度は下の図のように変化すると考えられます。

![輝度のグラフ1](images/10/05.png)

このある点 $x$ における色の変化量を求めるためには、微分式を使って $x$ における勾配を求める必要があります。
しかし、位置と輝度の関係を式で表せないと単純に勾配は計算できません。
そこで、代わりに $x$ を起点に、両隣の点、左隣の点を $x-1$、対応する輝度の値を $f(x-1)$、右隣の点を $x+1$、対応する輝度の値を $f(x+1)$ とします。

![輝度のグラフ1](images/10/06.png)

左右 ２ 点を通る直線というのは、$x$ における接線と似た形になります。このことから、左右 2 点を通る直線の傾きを求めることで、近似的に $x$ における接線の傾きを求めます。直線の傾きは次のようになります。

$$
\begin{align}
\begin{aligned} a & = \frac { f ( x + 1 ) - f ( x - 1 ) } { ( x + 1 ) - ( x - 1 ) } \\ & = \frac { f ( x + 1 ) - f ( x - 1 ) } { 2 } \end{aligned}
\end{align}
$$

求めたこの傾きの値とフィルタの値の関係はどのようになっているでしょうか。フィルタの計算を少し抽象的な形で考えてみます。グラフを使用したとき同様、着目したい点における輝度を $f(x)$、左右の値をそれぞれ $f(x-1)$、$f(x+1)$ とします。

![エッジ検出のフィルタ](images/10/07.png)

この時、計算結果は以下のようになります。

$$
\begin{align}
\begin{array} { l } { - 1 \times f \left( x-1 \right) + 0 \times f ( x ) + 1 \times f \left( x+1  \right) } { = f \left( x+1 \right) - f \left( x- 1 \right) } \end{array}
\end{align}
$$

$f(x+1) - f(x-1)$ という式は先程 2 点を通過する直線の傾きを計算した際の分子の値と一致します。分母の値は常に $2$ であり、固定値となりますので、分子の値のみに注目すればエッジ検出には十分な情報が得られるわけです。フィルタの計算は 2 点を通る直線の傾き、つまり勾配を近似していることがわかります。このようにフィルタの値を工夫することで、エッジを含む任意の特徴を抽出することが可能だと考えることができます。

## 画像処理の実装

この節では上で学んだ内容を Pillow（``PIL``）と NumPy（``numpy``）を使いながらコードで実装することで理解を深めていきます。具体的には、画像の読み込み、輝度の確認、チャンネルの操作やエッジ検出を実際に書いていきます。

### 画像のダウンロード

まず、今回使うサンプル画像をダウンロードします。

In [1]:
!wget https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png

### 画像の読み込み

Pillow を使って画像を読み込みます。
Pillow では次のようにファイルパスがわかっていれば一行で画像を読み込むことができます。

In [2]:
from PIL import Image

# 先程ダウンロードしたファイル名を指定
img_filepath = 'Lenna_(test_image).png'

# ファイルを読み込む
img = Image.open(img_filepath)
img

読み込んだ画像を NumPy の ``numpy.ndarray`` に変換することで配列として扱うことも可能です。ここではその変換を利用して画像のメタデータを確認します。

In [3]:
import numpy as np

# 変換とクラスの確認
img_array = np.asarray(img)
type(img_array)

In [4]:
# 高さ、幅、チャンネル数の確認
img_array.shape

In [5]:
# 詳細なデータ型の確認
img_array.dtype

画像の高さと幅、RGB に相当する 3 つチャンネルが存在することがわかります。 
データの型の ``uint8``は unsigned integer の略です。符号なし（非負の値のみ）の 8 ビット整数です。

### グレースケール変換

Pillow を使えば簡単にグレースケールに変換することもできます。

In [6]:
# グレースケールに変換
img_grayscale = img.convert('L')
img_grayscale

In [7]:
img_ndarray = np.asarray(img_grayscale)
img_ndarray.shape

グレースケールに変換すると色の情報が失われ、Pillow の仕様により、チャンネルに相当する軸が消えているのがわかります。厳密には、光度を表す値のみが情報として残ります。

### 畳み込み演算

次に上の節のフィルタを再現し、グレースケール画像に適用してみます。

In [8]:
from PIL import ImageFilter

# エッジ検出用のフィルタ（カーネル）
kernel = [
    -1, 0, 1,
    -1, 0, 1,
    -1, 0, 1
]

# フィルタの大きさ
kernel_size = (3, 3)

edge_filter = ImageFilter.Kernel(size=kernel_size, kernel=kernel, scale=1.0)

# フィルタで畳み込み演算を行う
img_edges = img_grayscale.filter(edge_filter)
img_edges

ここで出力された画像から確認できるように、例として挙げたフィルタは横方向の変化に強い、縦のエッジを検出するものであったことがわかりました。値を変えることで、縦や斜め方向に色の変化の強いエッジを検出する様々なフィルタを作ることも可能です。ちなみに、Pillow にはエッジ検出用のフィルタを含め、既にいくつかのフィルタが用意されています。エッジ検出用のフィルタに関しては値がここで紹介したものと違うので、検出されるエッジは異なりますが、同じように畳み込み演算を行なっていることに変わりはありません。

In [9]:
# Pillow で用意されているエッジ検出用フィルタを適用
img_edges_pillow = img_grayscale.filter(ImageFilter.FIND_EDGES)
img_edges_pillow

## Convolutional Neural Network（CNN）

上で紹介したフィルタや畳み込み演算は、コンピュータビジョンを始め様々な分野で近年優れた精度を示している CNN の基礎となります。この節では CNN とは何なのか、畳み込み演算やフィルタとどのような関係があるについて学んでいきます。

### CNN の概要

CNN はニューラルネットワークの一種で、**畳み込み層**を含むものを指します。畳み込み層は、全結合型で使う層とは違い、**フィルタを訓練**することによって画像のような高次元データ（主に 2 か 3 次元）に対して強みを発揮するニューラルネットワークのコンポーネントです。一言で CNN と言っても多種多様で、畳み込み層を含むモデルには色々な亜種があり、組み合わせによっては無限に存在します。この節では、まず歴史的な背景、一般的な構造の CNN と畳み込み層に焦点を当てます。

### ブレークスルー

#### 従来のコンピュータビジョン

コンピュータビジョンのタスクに、画像分類や物体認識があります。例えば、ネコとイヌの画像を分類するというタスクを考えます。
従来の手法では、前述したフィルタを用いて画像から分類するために必要な情報を抽出します。エッジの検出やグレースケール変換、ノイズの除去と言った処理を施し、情報を抽出していきます。この時に使うフィルタの値、又、手法の組み合わせは人間の今までの経験を元に考えられていました。ネコを判別をするのにネコの形をうまく捉えるような情報が必要だ、それならばこのエッジ検出器を使おう、といった流れです。
この特徴（特徴ベクトル）と support vector machine (SVM) のような識別器を使ってネコと認識できるようなモデルを学習するのが従来のアプローチとしては一般的でした。

![従来の画像分類](images/10/08.png)

#### CNN を使った画像分類

しかし、ネコの画像と言ってもパターンは無限に存在します。これらを網羅することは上の方法ではほぼ不可能です。この問題を解決したのが、画像認識のコンペティションの ImageNet Large Scale Visual Recognition Competition (ILSVRC) で 2012 年に優勝した AlexNet と呼ばれる CNN です。従来とは全く異なった手法で、今までの記録（精度）を大きく上回ったことをきっかけに、CNN が画像分類を含めコンピュータビジョンで多く活用されるようになりました。

### フィルタが訓練対象

AlexNet がどのように従来の手法と違ったのかと言うと画像の特徴を抽出するフィルタ自体を訓練対象のパラメータとしてモデル化したことです。つまり、ネコとイヌを分類するために本当に必要なフィルタを誤差逆伝播法で求めるので、人間が決めたフィルタを一切必要としません。このパラメータを保持するのが畳み込み層で、全結合層と同じように組み合わせることによってニューラルネットワークを構築します。

![CNNのイメージ](images/10/09.png)

### CNN の一般的な構造

一般的な CNN は畳み込み層を含め、大きく分けて以下の 3 種類の層から構築されます。

- 畳み込み（convolution）
- プーリング（pooling）
- 全結合（fully-connected）

順を追って確認します。

![CNNの構造](images/10/10.png)

#### 畳み込み

CNN の核となるのが畳み込みになります。これは前述したフィルタの計算を行う部分であり、特徴抽出を行います。フィルタの値は全てパラメータとなっていて、ランダムな値で初期化され、訓練を通して最適化されます。フィルタの数、フィルタのサイズ、ストライドの値、パディングの有無はハイパーパラメータとなっています。フィルタの数は、層の入力画像のチャンネル数と出力画像のチャンネル数に依存し、これらの積になります。各フィルタが一つの特徴を抽出するように最適化されることが期待されるので、出力のチャンネル数を入力のチャンネル数より大きく設定することで、次の層へより多い情報を伝搬できるようにハイパーパラメータを設定することが多いです。畳み込み演算を画像に施すと厳密には画像ではなくなるので、畳み込み層の出力を**特徴マップ**と呼ふことがあります。

#### プーリング

畳み込み層が出力する特徴マップには、省いても良い情報が含まれていると考えられます。例えば、エッジ検出の例では、反応が強い値だけどを伝搬することができれば、特徴抽出器としては十分だったはずです。コンピュータのリソースは有限です。なので、必要な情報のみを取り出すことで各特徴マップのサイズを小さくすることができれば、フィルタの数を増やし、より多くの特徴を抽出することができます。そこで**プーリング**を使います。
プーリングは畳み込み演算と似た操作で特徴マップの縮小を行います。具体的には、カーネルをストライドしながら特徴を抽出します。この時、ストライドを大きく設けることで出力の特徴マップを小さくします。更に、プーリングには次に続く畳み込み演算のフィルタの**受容野（receptive field）**を拡大するという効果もあります。受容野とは、フィルタの位置を固定したときに、重なる特徴マップの部分に対応している入力の画像（入力層へ与える画像）の範囲のことを指します。受容野が広がると、よりマクロで抽象的な特徴を一つのフィルタで獲得することができます。プーリングの処理も畳み込み層と同じようにモデルの一部として捉えますが、重要な違いは訓練対象のパラメータを保持していないと言う点です。パラメータを持たない代わりに、プーリングにはいくつかの種類が存在します。代表的なものに**マックスプーリング（max pooling）**や**アベレージプーリング（average pooling）**があります。前者は特徴マップからカーネルの範囲内の、最大値だけを伝搬し、後者は平均値を伝搬します。下のマックスプーリングの
図ではカーネルのサイズとストライドを同じに設定することで特徴マップを縮小しています。一般的なハイパーパラメータの設定です。

![MaxPooling](images/10/11.png)

#### 全結合（fully-connected）

モデルを定義する際、畳み込み層とプーリング層は組みにして、隠れ層として何重にも重ねることが多いです。畳み込みとプーリングの操作を複数回行った後、最終的に抽出される複数のチャンネルからなる特徴マップを 1 列に並べ、全結合層に入力として与えることで、例えば予測を行うことが可能です。

本章では、基礎的な画像処理や CNN について学びました。

次章では CNN の実装方法をお伝えします。